In [1]:
import os
import torch 
import numpy
import pandas as pd 
import cudf
import cupy
import nvtabular as nvt

### Create random input data similar to pre-processed Yoochoose dataset structure 

In [2]:
NUM_ROWS = 10000
session_length = 20
inputs = {
    'session_id': numpy.random.randint(70000, 80000, NUM_ROWS),
    'day': numpy.random.randint(1, 10, NUM_ROWS),
    'item_id': numpy.random.randint(1, 51996, NUM_ROWS),
    'category': numpy.random.randint(0, 332, NUM_ROWS),
    'timestamp/age_days': numpy.random.uniform(0, 1, NUM_ROWS),
    'timestamp/weekday/sin' : numpy.random.uniform(0, 1, NUM_ROWS),
    'purchase': numpy.random.randint(0, 2, NUM_ROWS)
    }
random_data = cudf.DataFrame(inputs)
random_data

,session_id,day,item_id,category,timestamp/age_days,timestamp/weekday/sin,purchase
0,73335,1,25204,135,0.283280,0.126510,0
1,77282,2,20726,153,0.777557,0.167179,0
2,73655,5,47016,327,0.807628,0.094816,0
3,73205,6,8130,102,0.581975,0.374510,0
4,72327,4,34756,59,0.280593,0.909303,0
...,...,...,...,...,...,...,...
9995,70511,6,14203,40,0.408895,0.757879,1
9996,78436,4,25377,85,0.028902,0.144950,0
9997,73295,3,26406,300,0.732575,0.849151,1
9998,75522,9,4956,231,0.186765,0.603853,1


### NVTabular workflow

- #TODO : Change the workflow using tagging API once it is finalized  

In [3]:
# Define Groupby Workflow
groupby_features = list(inputs.keys()) >> nvt.ops.Groupby(
    groupby_cols=["session_id"], 
    aggs={
        "item_id": ["list"],
        "category": ["list"],     
        "day": ["first"],
        "purchase": ["first"],
        "timestamp/age_days": ["list"],
        'timestamp/weekday/sin': ["list"],
        },
    name_sep="-")
# Trim sessions to first 20 items 
groupby_features_list = [x for x in groupby_features.output_columns.names if '-list' in x]

#groupby_features_nonlist:  need to fix a BUG related to adding two workflow nodes
#groupby_features_trim = groupby_features_list >> nvt.ops.ListSlice(0,20) >> nvt.ops.Rename(postfix = '_trim')
workflow = nvt.Workflow(groupby_features)
dataset = nvt.Dataset(random_data, cpu=False)
workflow.fit(dataset)
sessions_gdf = workflow.transform(dataset).to_ddf().compute()

# Re-compute tri: to be removed when the BUG of two workflow nodes is fixed : 
groupby_features_trim =  groupby_features_list >> nvt.ops.ListSlice(0,20) >> nvt.ops.Rename(postfix = '_trim')
workflow = nvt.Workflow(list(sessions_gdf.columns)  + groupby_features_trim)
dataset = nvt.Dataset(sessions_gdf, cpu=False)
workflow.fit(dataset)
sessions_gdf = workflow.transform(dataset).to_ddf().compute()

In [4]:
sessions_gdf.head(3)

,category-list_trim,timestamp/age_days-list_trim,item_id-list_trim,timestamp/weekday/sin-list_trim,purchase-first,category-list,session_id,timestamp/age_days-list,day-first,item_id-list,timestamp/weekday/sin-list
0,[207],[0.5123948838105032],[39511],[0.7095749236097595],0,[207],70000,[0.5123948838105032],4,[39511],[0.7095749236097595]
1,[164],[0.8770186544541125],[51728],[0.2572932168328079],0,[164],70002,[0.8770186544541125],4,[51728],[0.2572932168328079]
2,[279],[0.9985119530693698],[19247],[0.016013178790216687],1,[279],70003,[0.9985119530693698],1,[19247],[0.016013178790216687]


- We can save the workflow.

In [5]:
workflow.save('workflow_inference_test')

### Export pre-processed data by day 

In [6]:
# requires cudf + cupy + nvtabular + dask_cudf
from transformers4rec.utils.processing_utils import save_time_based_splits
save_time_based_splits(data=nvt.Dataset(sessions_gdf),
                       output_dir= "./preproc_sessions_by_day_ts",
                       partition_col='day-first',
                       timestamp_col='session_id', 
                      )

Creating time-based splits: 100%|██████████| 9/9 [00:03<00:00,  2.53it/s]


# Transformers4rec model  

In [7]:
import torch 
import transformers4rec.torch as torch4rec
from transformers4rec.torch import TabularSequenceFeatures, MLPBlock, SequentialBlock, Head, TransformerBlock

from transformers4rec.utils.schema import DatasetSchema
from transformers4rec.torch.head import NextItemPredictionTask
from transformers4rec.config import transformer
from transformers4rec.torch.ranking_metric import NDCGAt, AvgPrecisionAt, RecallAt

- Manually set the schema 

In [8]:
# Define schema object to pass it to the SequentialTabularFeatures
schema = DatasetSchema.from_schema("schema.pb")

### Define the sequential input module

Below we define our `input` bloc using [`SequentialTabularFeatures` class](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/torch/features/sequential.py). The `from_schema` module directly parse schema and accepts categorical and continuous sequential inputs and supports data augmentation, data aggregation, `sequential-concat` and `elementwise-sum` aggregations, the projection of the interaction embeddings and the masking tasks.

`max_sequence_length` defines the maximum sequence length of our sequential input, and if `continuous_projection` argument is set,  all numerical features are concatenated and projected by a number of MLP layers.

In [9]:
inputs = TabularSequenceFeatures.from_schema(
        schema,
        max_sequence_length=20,
        continuous_projection=64,
        d_output=100,
        masking="causal",
    )

### End-to-end session-based Transformer-based model for item prediction:

- LM task + HF Transformer architecture + Next item-prediction task. 
- We build a [T4RecConfig](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/config/transformer.py#L8) class to update the config class of the transformer architecture with the specified arguments, then load the related model. Here we use it to instantiate an XLNET model according to the  arguments (d_model, n_head, etc.), defining the model architecture.
- [TransformerBlock](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/torch/block/transformer.py#L37) class is created to support HF Transformers for session-based and sequential-based recommendation models.
- [NextItemPredictionTask](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/torch/head.py#L212) is the class to support next item prediction task.

In [10]:
#### Define XLNetConfig class and set default parameters 
# Set HF config of XLNet 
transformer_config = transformer.XLNetConfig.build(
    d_model=64, n_head=4, n_layer=2, total_seq_length=20
)
# Define the model block including: inputs, masking, projection and transformer block.
body = torch4rec.SequentialBlock(
    inputs, torch4rec.MLPBlock([64]), torch4rec.TransformerBlock(transformer_config, masking=inputs.masking)
)

# Define the head related to next item prediction task 
head = torch4rec.Head(
    body,
    torch4rec.NextItemPredictionTask(weight_tying=True, hf_format=True),
    inputs=inputs,
)

# Get the end-to-end Model class 
model = torch4rec.Model(head)

# Training and evaluation

- **Set Training arguments**

In [11]:
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch import Trainer
#Set argumentd for training 
train_args = T4RecTrainingArguments(local_rank = -1, dataloader_drop_last = True, data_loader_engine='pyarrow',
                                  report_to = [], debug = ["r"], gradient_accumulation_steps = 32,
                                  per_device_train_batch_size = 512, per_device_eval_batch_size = 32,
                                  output_dir = ".", lr_scheduler_type='cosine', 
                                  learning_rate_num_cosine_cycles_by_epoch=1.5,
                                  max_sequence_length=20, fp16=False, no_cuda=True)

- **Define paths to train and eval data**

In [12]:
import glob
train_data_paths = glob.glob("./preproc_sessions_by_day_ts/*/train.parquet")[:-1]
eval_data_paths = glob.glob("./preproc_sessions_by_day_ts/*/valid.parquet")[:-1]

- **Define Trainer**

In [13]:
# Instantiate the T4Rec Trainer, which manages training and evaluation
trainer = Trainer(
    model=model,
    args=train_args,
    schema=schema,
    compute_metrics=True,
    train_dataset_or_path=train_data_paths,
    eval_dataset_or_path=eval_data_paths,
)

- **Train the model**  

In [14]:
trainer.reset_lr_scheduler()
trainer.train()

Step,Training Loss


TrainOutput(global_step=3, training_loss=3.424956957499186, metrics={'train_runtime': 64.8283, 'train_samples_per_second': 0.046, 'total_flos': 0.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 172032, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 2959847424, 'train_mem_gpu_alloc_delta': 0, 'train_mem_cpu_peaked_delta': 341401600, 'train_mem_gpu_peaked_delta': 0})

- **Compute evaluation metrics**

In [15]:
eval_metrics = trainer.evaluate(eval_dataset=eval_data_paths, metric_key_prefix='eval')
for key in sorted(eval_metrics.keys()):
    print("  %s = %s" % (key, str(eval_metrics[key])))

  epoch = 3.0
  eval_avgprecisionat_10 = 0.0
  eval_avgprecisionat_20 = 0.0
  eval_loss = 10.94094467163086
  eval_mem_cpu_alloc_delta = 26046464
  eval_mem_cpu_peaked_delta = 40960
  eval_mem_gpu_alloc_delta = 0
  eval_mem_gpu_peaked_delta = 0
  eval_ndcgat_10 = 0.0
  eval_ndcgat_20 = 0.0
  eval_recallat_10 = 0.0
  eval_recallat_20 = 0.0
  eval_runtime = 12.7475
  eval_samples_per_second = 42.675


- **Compute Train metrics**

In [16]:
train_metrics = trainer.evaluate(eval_dataset=train_data_paths, metric_key_prefix='train')
for key in sorted(train_metrics.keys()):
    print("  %s = %s" % (key, str(train_metrics[key])))

  epoch = 3.0
  eval_mem_cpu_alloc_delta = 5509120
  eval_mem_cpu_peaked_delta = 286720
  eval_mem_gpu_alloc_delta = 0
  eval_mem_gpu_peaked_delta = 0
  train_avgprecisionat_10 = 0.0
  train_avgprecisionat_20 = 0.0
  train_loss = 10.914937973022461
  train_ndcgat_10 = 0.0
  train_ndcgat_20 = 0.0
  train_recallat_10 = 0.0
  train_recallat_20 = 0.0
  train_runtime = 11.7674
  train_samples_per_second = 54.387


* **Save the model**

In [ ]:
trainer._save_model_and_checkpoint()

* **Reload model:**

In [ ]:
trainer.load_model_trainer_states_from_checkpoint('./checkpoint-%s'%trainer.state.global_step)

- **Re-compute eval metrics of train data**

In [ ]:
train_metrics = trainer.evaluate(eval_dataset=train_data_paths, metric_key_prefix='train')
for key in sorted(train_metrics.keys()):
    print("  %s = %s" % (key, str(train_metrics[key])))

* **Resume Training** 

In [ ]:
# reset lr scheduler to train on new day data
trainer.reset_lr_scheduler()
# set new data from last day
trainer.train_dataset = train_data_paths[-1]
trainer.train(resume_from_checkpoint='./checkpoint-%s'%trainer.state.global_step)

- **Evaluate on last day**

In [ ]:
# set new data from last day
eval_metrics = trainer.evaluate(eval_dataset=eval_data_paths[-1], metric_key_prefix='eval')
for key in sorted(eval_metrics.keys()):
    print("  %s = %s" % (key, str(eval_metrics[key])))

# Incremental Training over a list of time windows 

In [ ]:
# Instantiate the T4Rec Trainer, which manages training and evaluation
trainer = Trainer(
    model=model,
    args=train_args,
    schema=schema,
    compute_metrics=True,
)

In [ ]:
start_time_window_index = 1
final_time_window_index = 7
for time_index in range(start_time_window_index, final_time_window_index):
    # Set data 
    time_index_train = time_index
    time_index_eval = time_index + 1
    train_paths = glob.glob(f"./preproc_sessions_by_day_ts/{time_index_train}/train.parquet")
    eval_paths = glob.glob(f"./preproc_sessions_by_day_ts/{time_index_eval}/valid.parquet")  
    
    # Train on day related to time_index 
    print('*'*20)
    print("Launch training for day %s are:" %time_index)
    print('*'*20 + '\n')
    trainer.train_dataset = train_paths
    trainer.reset_lr_scheduler()
    trainer.train()
    trainer.state.global_step +=1
    # Evaluate on the following day
    trainer.eval_dataset = eval_paths
    train_metrics = trainer.evaluate(metric_key_prefix='eval')
    print('*'*20)
    print("Eval results for day %s are:\t" %time_index_eval)
    print('\n' + '*'*20 + '\n')
    for key in sorted(train_metrics.keys()):
        print(" %s = %s" % (key, str(train_metrics[key]))) 
    trainer.wipe_memory()